In [ ]:
import qdrant_client
from qdrant_client.http import models

### Notebook prise en main Qdrant

L'objectif de ce notebook est de proposer un tutoriel dans lequel on retrouve toutes les opérations relatives à la gestion d'un index

In [ ]:
# Initialisation du client
client_qdrant = qdrant_client.QdrantClient(
    "http://localhost:6333",
)

In [ ]:
# Affichage des données
data = client_qdrant.scroll(
    collection_name="qdr",
    limit=1000, # Conditionne le nombre de ligne afficher
    with_payload=True,
    with_vectors=True,
)

Dans Qdrant, le plus haut niveau d'abstraction s'effectue au niveau du concept de "collection". Cela permet d'instancier par exemple des instances de dev, de preprod et de prod pour une application sur une même instance Qdrant. Il n'est pas recommander par Qdrant d'utiliser les collections comme des bases de données car leur multiplication pourrait entraîner une baisse de performance. 

In [ ]:
client_qdrant.create_collection(
    collection_name="qdr",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE),
    hnsw_config=models.HnswConfigDiff(
        payload_m=16,
        m=0,
    ),
)

Ici, on vient créer une collection supportant le multitenancy i.e la création de plusieurs indexs de manière optimisée

In [ ]:
# Récupération des noms des collections
collections_name  = [c.name for c in client_qdrant.get_collections().collections]

In [ ]:
collections_name

Qdrant se dote d'un deuxième niveau d'abstraction : les index. C'est eux qui correspondent à notre collection dans l'application CARADOC.

In [ ]:
client_qdrant.create_payload_index(
    collection_name="demo",
    field_name="datascience",
    field_schema=models.PayloadSchemaType.KEYWORD,
)


In [ ]:
client_qdrant.get_collection("demo")

In [ ]:
# ajout de données 
from qdrant_client.http.models import PointStruct
operation_info = client_qdrant.upsert(
    collection_name="demo",
    wait=True,
    points=[
        PointStruct(id=1, vector=[0 for i in range(1024)], payload={"filename": "file1", "content":"part1", "index":"token"}),
        PointStruct(id=2, vector=[0 for i in range(1024)], payload={"filename": "file1", "content": "part2", "index": "token"}),
        PointStruct(id=3, vector=[0 for i in range(1024)], payload={"filename": "file2", "content":"part1", "index":"token"}),
    ],
)

In [ ]:
# Affichage des données
data = client_qdrant.scroll(
    collection_name="qdr",
    limit=1000, # Conditionne le nombre de ligne afficher
    with_payload=True,
    with_vectors=True,
)

In [ ]:
len(data[0][3].vector)

In [ ]:
def clean_qdrant_index(index: str, collection_name: str = "demo"):
    """Function to clean user session document on Qdran

    Args:
        token (str): User token
        collection_name (str, optional): Collection name in Qdrant. Defaults to "ayd_dev".
    """
    try:
        client_qdrant.delete(
            collection_name=collection_name,
            points_selector=models.FilterSelector(
                filter=models.Filter(
                    must=[
                        models.FieldCondition(key="index", match=models.MatchValue(value=str(index))),
                    ],
                )
            ),
        )
    except Exception as e:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail=f"An error occurred while removing sessions files in Qdrant - Error message: {e}",
        )

In [ ]:
clean_qdrant_index(index="datascience")

In [ ]:
# Suppression de la collection
client_qdrant.delete_collection("qdr")

In [ ]:
def clean_file_qdrant_index(index: str, filename: str):
    """
    Removes a file from a specific qdrant index
    :param index:
    :param filename:
    :return:
    """
    try:
        client_qdrant.delete(
            collection_name="demo",
            points_selector=models.FilterSelector(
                filter=models.Filter(
                    must=[
                        models.FieldCondition(
                            key="index",
                            match=models.MatchValue(value=str(index)),
                        ),
                        models.FieldCondition(
                            key="filename",
                            match=models.MatchValue(value=str(filename)),
                        ),
                    ],
                )
            ),
        )
    except Exception as e:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail=f"An error occurred while removing sessions files in Qdrant - Error message: {e}",
        )


In [ ]:
clean_file_qdrant_index(index="token", filename="file1")

In [ ]:
from pymongo import MongoClient

In [ ]:
mongo_client = MongoClient(
        host = [ str("10.156.254.10") + ":" + str(27018) ],
        serverSelectionTimeoutMS = 3000, # 3 second timeout
        username = "dev",
        password = "dev",
    )


In [ ]:
database_names = mongo_client.list_database_names()

In [ ]:
mongo_client.get_database('local')